<h1>Chapter 10. Implementing Design Patterns with Full-Fledged Functions</h1>

<h2>Practical Example: Reworking a Pattern Strategy</h2>

<h3>Classic Strategy</h3>

The Strategy pattern allows clients to select from a range of interchangeable algorithms during runtime, promoting flexibility and maintainability. It consists of three main components:</br>
**Context**: This class maintains a reference to the current strategy and collaborates with it, providing an interface for accessing strategy algorithms.</br>
**Strategy**: This interface or abstract class defines the contract for all supported algorithms, specifying common methods that concrete strategies must implement.</br>
**Concrete Strategies**: These are individual algorithm implementations that adhere to the Strategy interface, providing specific implementations for each algorithm.

Implementation of the `Order` class using interchangeable discounting strategies. 

In [1]:
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity


class Order(NamedTuple):  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        return f"<Order total: {self.total():.2f} due: {self.due():.2f}>"


class Promotion(ABC):  # the Strategy: an abstract base class
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a possitive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

In [2]:
# Two customers: Joe has 0 loyalty, Ann has 1100
joe = Customer(name='John Doe', fidelity=0)
ann = Customer(name='Ann Smith', fidelity=1100)

# One shopping card with three items
cart = (
    LineItem(product='banana', quantity=4, price=Decimal('.5')),
    LineItem(product='apple', quantity=10, price=Decimal('1.5')),
    LineItem(product='watermelon', quantity=5, price=Decimal(5)),
)

# The banana_cart contains 30 bananas and 10 apples
banana_cart = (
    LineItem(product='banana', quantity=30, price=Decimal('.5')),
    LineItem(product='apple', quantity=10, price=Decimal('1.5')),
)

# There are different items in the long_oreder that cost 1.00$ each
long_order = tuple(
    LineItem(product=str(sku), quantity=1, price=Decimal(1)) for sku in range(10)
)

In [5]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [6]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [7]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [8]:
Order(ann, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [9]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [10]:
Order(ann, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>